In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import cross_validate
from tqdm import tnrange, tqdm_notebook, tqdm
from datetime import timedelta
from datetime import datetime
from sklearn import metrics

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import scipy.stats as stats
import xgboost as xgb
import requests as r
import pandas as pd
import seaborn as s
import numpy as np
import googlemaps
import postgres
import holidays
import config
import json
import math
import gc

# enable automatic garbage collection
gc.enable()

ie_holidays = holidays.Ireland()

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Importing Data

### Stop Distances

In [ ]:
stop_distances = pd.read_csv("stored_queries/distancedata.csv", header=None)
stop_distances.columns = ['stopid','previous_stopid','distance']
stop_distances.head()

### Stops

In [ ]:
stops = pd.read_csv("stop_information.csv")
cols = list(stops.columns)
cols[0] = 'ix'
stops.columns = cols
stops.drop(columns=cols[0], inplace=True)

### weather Data Preprocessing

In [ ]:
weather = pd.read_csv("stored_queries/weather.csv")
weather.icon = weather.icon.astype('category')
weather.weatherdayofservice = pd.to_datetime(weather.weatherdayofservice)

### Leavetimes Data Preprocessing

In [ ]:
data = pd.read_csv("stored_queries/combined.csv")
data.columns = ['dayofservice','tripid','lineid','direction','progrnumber','stopid','plannedDEP','plannedARR','actualDEP','actualARR','routeid']
gc.collect()

data.drop(columns=['routeid','plannedDEP','plannedARR','actualDEP'], inplace=True)
data.drop_duplicates()
gc.collect()

# data.dayofservice = pd.to_datetime(data.dayofservice.loc[:])
gc.collect()

data.sort_values(by=['dayofservice','lineid','tripid','direction','progrnumber'], inplace=True)
gc.collect()

data.drop(columns=['lineid'], inplace=True)

# data.to_csv("stored_queries/combinedsorted.csv", index=False, chunksize=500000)

# del data
# gc.collect()

In [ ]:
data.to_csv("stored_queries/combinedsorted.csv", index=False, chunksize=500000)

del data
gc.collect()

In [ ]:
data = pd.read_csv("stored_queries/combinedsorted.csv")
data.head()

In [ ]:
data.dayofservice = pd.to_datetime(data.dayofservice.loc[:])

**Remove Stops which are no longer in service**

In [ ]:
# ==================== REMOVE INACTIVE STOPS ====================== #
active_stopids = stops.stopid.values

# remove all inactive stops from the dataset. -> additional models that arent needed. 
data = data[data.stopid.isin(active_stopids)]

**Make Arrays for previous stops etc..**

In [ ]:
# previous stopid
previousstops =  list(data.stopid)
previousstops = np.array(previousstops[:-1]).astype(int)

In [ ]:
# progrnumber of previous stopid
previousstops_progrnumber = list(data.progrnumber)
previousstops_progrnumber = np.array(previousstops_progrnumber[:-1]).astype(int)

In [ ]:
# Actual arrival time of previous stopid
previousstops_actualARR = list(data.actualARR)
previousstops_actualARR = np.array(previousstops_actualARR[:-1])

In [ ]:
# Delete the first row of the dataframe to shift the progrnumbers by one. 
data = data.iloc[1:]

In [ ]:
data['previous_stopid'] = previousstops
data['previous_stopARR'] = previousstops_actualARR
data['previous_progrnumber'] = previousstops_progrnumber

In [ ]:
data = data[data.progrnumber != 1]
data.dropna(inplace=True);

In [ ]:
# recast type of integer cols from float to int. 
data.previous_stopid = data.previous_stopid.astype(int)
data.previous_progrnumber = data.previous_progrnumber.astype(int)

In [ ]:
# make progrnumber difference column and then drop anything thats not exactly 1, removes data which skips stops. 
data['progrnumber_difference'] = data.progrnumber - data.previous_progrnumber

# remove non-consecutive stop pairs.
data = data[data.progrnumber_difference==1]

In [ ]:
data = data[['dayofservice', 'tripid','stopid', 'previous_stopid', 'actualARR', 'previous_stopARR']]

In [ ]:
data.head()

In [ ]:
data.to_csv("stored_queries/combinedfinal.csv", index=False, chunksize=500000)

In [ ]:
data.dtypes

## import ready to go leavetimes table

In [ ]:
data = pd.read_csv("stored_queries/combinedfinal.csv")

In [ ]:
data.dayofservice = pd.to_datetime(data.dayofservice.loc[:])

In [ ]:
data.dtypes
data.head()

## Separating stop_pairs

In [ ]:
stop_pairs = data[['stopid','previous_stopid']].drop_duplicates()

print("There are %d unique pairs of stops in stop distances" % (stop_pairs.count()[0]))

## Loop through stop_pairs

In [ ]:
no_stops = stop_pairs.count()[0]
print("There are %d models to train." % no_stops)

## Model Training 

In [ ]:
def set_season(x):
    winter = [11,12,1]
    autumn = [10,9,8]
    spring = [4,3,2]

    if x in winter:
        return 'Winter'
    elif x in autumn:
        return 'Autumn'
    elif x in spring:
        return 'Spring'
    else:
        return 'Summer'

In [ ]:
target     = ['travel_time']

predictors = ['temperature','humidity', 'windSpeed', 'rain', 'hour', 'holiday', 'weekend',
              'month','season_Winter','season_Autumn','season_Summer','season_Spring',
              'icon_clear-day', 'icon_clear-night', 'icon_cloudy', 'icon_fog',
              'icon_partly-cloudy-day', 'icon_partly-cloudy-night', 'icon_rain','icon_wind']

In [ ]:
models = dict()
emptys = []
metric = dict()
Error_routes = []


# Model Training parameters
param = {
    'tree_method':'gpu_hist',
    'eta': 0.15,
    'max_depth': 6
}

num_rounds = 10000

for pair in tqdm_notebook(stop_pairs.iterrows(), desc="Progress: ", total=no_stops):
    
    # Travelling From A -> B
    A = pair[1][1]
    B = pair[1][0]
    
    # ========================= Rows from A -> B =========================== #
    sample = data[(data.stopid==B)&(data.previous_stopid==A)]
        
    # ========================= Adding Weather Data ======================== #    
    sample.actualARR = sample.dayofservice + pd.to_timedelta(sample.actualARR, unit = 'seconds') # in nanoseconds
    sample.previous_stopARR = sample.dayofservice + pd.to_timedelta(sample.previous_stopARR, unit = 'seconds') # in nanoseconds

    # new columns for combining
    sample['weather_merge_time'] = sample.actualARR.dt.round('H') #  .dt useful
    sample.sort_values(by='weather_merge_time', inplace=True)

    # weather data
    weather.weatherdayofservice = weather.weatherdayofservice + pd.to_timedelta(weather.hour, unit='hour')
    weather.sort_values(by='weatherdayofservice', inplace=True)
    
    # using merge_asof to save data. getting nearest weather. 
    combinedata = pd.merge_asof(sample,
                                weather[['icon',
                                         'temperature',
                                         'humidity',
                                         'windSpeed',
                                         'rain',
                                         'weatherdayofservice',
                                         'hour']], 
                                left_on='weather_merge_time', 
                                right_on='weatherdayofservice', 
                                direction='nearest')
    
    combinedata.drop(columns=['weather_merge_time','weatherdayofservice'], inplace=True)
    
    # ======================== Adding Holiday ============================= #
    combinedata['holiday'] = combinedata.dayofservice.apply(lambda x: x in ie_holidays)
    
    # ======================== Adding weekend ============================= #
    combinedata['weekend'] = combinedata.dayofservice.dt.weekday.isin([5,6])
    
    # ======================== Adding month   ============================= #
    combinedata['month'] = combinedata.dayofservice.dt.month

    # ======================== Adding season  ============================= #
    combinedata['season'] = combinedata.dayofservice.dt.month.apply(set_season)
    
    # ======================== Adding travel_time  ============================= #    
    combinedata['travel_time'] = (pd.to_timedelta(combinedata.actualARR) - 
                                  pd.to_timedelta(combinedata.previous_stopARR)).dt.seconds
    
    # drop any values less than 5 seconds [assumed erroneous]
    combinedata = combinedata[combinedata.travel_time > 5]
    
    # ============== Encoding Categorical Data : Season  =================== #
    combinedata.season = combinedata.season.astype('category', categories=['Summer','Spring','Autumn','Winter'])

    combinedata = pd.concat([combinedata, pd.get_dummies(combinedata.season, prefix='season')], axis=1)
    combinedata.drop(columns=['season'], inplace=True)

    # ============== Encoding Categorical Data : Icons  ==================== #
    combinedata.icon = combinedata.icon.astype('category', categories=['partly-cloudy-day', 'partly-cloudy-night', 'clear-day', 'clear-night', 'rain', 'fog', 'cloudy', 'wind'])

    combinedata = pd.concat([combinedata, pd.get_dummies(combinedata.icon, prefix='icon')], axis=1)
    combinedata.drop(columns=['icon'], inplace=True)

    combinedata.dropna(inplace=True)
    
    modeldata = combinedata[['travel_time','temperature','humidity', 'windSpeed', 'rain', 'hour', 
                             'holiday', 'weekend','month','season_Winter','season_Autumn','season_Summer','season_Spring',
                             'icon_clear-day', 'icon_clear-night', 'icon_cloudy', 'icon_fog',
                             'icon_partly-cloudy-day', 'icon_partly-cloudy-night', 'icon_rain','icon_wind']]
    
    
    # ============================== TRAINING ============================== #
    
    
    if modeldata.count()[0] > 10:

        # ========================= Removing Outliers ========================== #
        travel_sigma = modeldata.travel_time.std()

        # Only allow travel times greater than zero 
        modeldata = modeldata[modeldata.travel_time >= 0]

        # Filter outliers from the dataset [ \mu - 3σ < x < 3.5σ + μ ] - skewed dataset
        modeldata = modeldata[modeldata.travel_time < modeldata.travel_time.mean() + 3.5*travel_sigma]
        modeldata = modeldata[modeldata.travel_time > modeldata.travel_time.mean() - 2*travel_sigma]

        # ========================= Remove Null Data =========================== #
        modeldata.dropna(inplace=True)

        # ========================= Test/Train Splits ========================== #
        X_train, X_test, y_train, y_test = train_test_split(modeldata[predictors],modeldata[target].values.ravel(), test_size=0.25, shuffle=True)

        # ========================== Making DMatrices ========================== #
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)

        # =========================== Training Model =========================== #

        try:

            # ============ Train ============= #
            model = xgb.train(param, dtrain, num_rounds, evals=[(dtest, 'Test')], verbose_eval=False, early_stopping_rounds=100)
            model.save_model(f"ModelFiles/StopModels/{A}_{B}.model")

            # ============ Testing Accuracy ========== #
            preds = model.predict(dtest)
            metric[f'{A}_{B}'] = dict()
            metric[f'{A}_{B}']['rmse'] = np.sqrt(metrics.mean_squared_error(preds, y_test))
            metric[f'{A}_{B}']['preds']= preds
            metric[f'{A}_{B}']['ytest']= y_test

        except Exception as e:
            print(f"Error with route: {A} -> {B}")
            Error_routes.append((A,B))

    else:
        print(f"Empty Set Error: {A} -> {B} | Count: {modeldata.count()[0]} | Pre-Count: {sample.count()[0]}")
        emptys.append((A,B))

In [ ]:
xgb.plot_importance(model)

In [ ]:
# run this to see how bad its fucked up m8
print(len(emptys))

rmses_array = []

for row in metric:
    rmses_array.append((metric[row]['rmse']))
    
    plt.figure()
    plt.plot(metric[row]['preds'])
    plt.plot(metric[row]['ytest'], alpha=0.1)

In [ ]:
# plt.plot(metric['226_228']['preds'])
plt.plot(metric['226_228']['ytest'] - metric['226_228']['preds'],alpha=0.5)


## Prediction function

In [7]:
import config
import json 
import numpy as np
import pandas as pd
import xgboost as xgb
import requests as req
import holidays as hol
ie_holidays = hol.Ireland()

import importlib
importlib.reload(config)

from datetime import date, time, datetime

def set_season(x):
    winter = [11,12,1]
    autumn = [10,9,8]
    spring = [4,3,2]

    if x in winter:
        return 'Winter'
    elif x in autumn:
        return 'Autumn'
    elif x in spring:
        return 'Spring'
    else:
        return 'Summer'
       

def prediction_route(PDate, StopA, StopB, PTime):
    """
    Return an estimate of travel time, in seconds, for a given journey. 
    
    inputs:
    ---------------------------------------
    (str) PDate:            YYYY-MM-DD
    (str) PTime:            HH:MM
    (str) StopA:            Start Stop
    (str) StopB:            End Stop
    
    
    Outputs:
    ---------------------------------------
    (int) Travel Time:          Seconds
    """
    
    # =========================== Import Model ========================= #
    
    print('here')
    
    model = xgb.Booster()
    model.load_model(f"ModelFiles/StopModels/{StopA}_{StopB}.model")
    
    print('here')
    
    # ====================== Dateand Time objects ====================== #
    
    ddate = date(int(PDate[:4]), int(PDate[5:7]), int(PDate[-2:]))
    dtime = time(int(PTime[:2]), int(PTime[-2:]))
        
    # ========================== Weather Data ========================== # 
    # will need to sync this with the automated live weather updates to not waste calls. Also add forecasting option. 
    
    weathercall = req.get(f"https://api.darksky.net/forecast/{config.darksky_api}/53.3498,-6.2603").content
    weather = json.loads(weathercall)
    weather= weather['currently']
    
    print('here')
    
    # ======================== Inputs DataFrame ======================== #
    
    predictors = ['temperature','humidity', 'windSpeed', 'rain', 'hour', 'holiday', 'weekend',
                  'month','season_Winter','season_Autumn','season_Summer','season_Spring',
                  'icon_clear-day', 'icon_clear-night', 'icon_cloudy', 'icon_fog',
                  'icon_partly-cloudy-day', 'icon_partly-cloudy-night', 'icon_rain','icon_wind']  
    
    # Make dataframe of inputs. 
    inputs = pd.DataFrame(np.zeros(len(predictors))).T
    inputs.columns = predictors
    
    inputs.hour = dtime.hour
    inputs.month= ddate.month  
    # ========================= Weather Columns ======================== #
    
    inputs.temperature = weather['temperature']
    inputs.humidity = weather['humidity']
    inputs.windSpeed = weather['windSpeed']
    
    # convert in inches of liquid water per hour to mm
    inputs.rain = float(weather['precipIntensity'])/0.0394
    
    # ========================= Weekday/Weekend ======================== #    
    
    if ddate.weekday() in [5,6]:
        inputs.weekday=False
    else:
        inputs.weekday=True 
    
    # ===================== One Hot Encoded Columns ==================== #  
    
    inputs["icon_{0}".format(weather['icon'])]=1
    inputs["season_{0}".format(set_season(ddate.month))]=1
    
    # ========================= Applying Model ========================= #
    
    print('here')
    
    inputdata = xgb.DMatrix(inputs)
    estimate = model.predict(inputdata)
    
    # ======================= Adjust for #stops ======================== #
    
    estimate = estimate.tolist()[0]
    
    # ========================= Returning Data ========================= #
    
    return estimate


In [ ]:
"""
Return an estimate of travel time, in seconds, for a given journey. 

inputs:
---------------------------------------
(str) PDate:            YYYY-MM-DD
(str) PTime:            HH:MM
(str) StopA:            Start Stop
(str) StopB:            End Stop
"""

PDate = "2019-08-15"
PTime = "10:00"
StopA = "226"
StopB = "228"

print(prediction_route(PDate, StopA, StopB, PTime))


here
here


In [2]:
weathercall = req.get(f"https://api.darksky.net/forecast/{config.darksky_api}/53.3498,-6.2603").content

In [3]:
weather=json.loads(weathercall)

In [6]:
weather['currently']

{'time': 1565858692,
 'summary': 'Mostly Cloudy',
 'icon': 'partly-cloudy-day',
 'nearestStormDistance': 71,
 'nearestStormBearing': 31,
 'precipIntensity': 0,
 'precipProbability': 0,
 'temperature': 59.75,
 'apparentTemperature': 59.75,
 'dewPoint': 50.56,
 'humidity': 0.72,
 'pressure': 1012.95,
 'windSpeed': 18.01,
 'windGust': 23.64,
 'windBearing': 279,
 'cloudCover': 0.51,
 'uvIndex': 2,
 'visibility': 6.216,
 'ozone': 349}